In [1]:
import numpy as np
import pandas as pd
import scanpy as scp

# Plotting imports
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
%matplotlib inline

In [14]:
week6_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week6")
week6_ad.var_names_make_unique()

week7_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week7")
week7_ad.var_names_make_unique()

week8_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week8")
week8_ad.var_names_make_unique()

week15_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week15")
week15_ad.var_names_make_unique()

week16_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week16")
week16_ad.var_names_make_unique()


In [15]:
week12_ad = scp.read_10x_mtx("/bigdata/faustmanlab/bch/sc-testis/human/gse143356/week12/Male_Week_12_gene_matrix/")
week12_ad.var_names_make_unique()
